In [1]:
import inspect
from time import sleep


from CybORG import CybORG
# from CybORG.Agents import B_lineAgent, BlueReactRestoreAgent, BlueReactRemoveAgent, SleepAgent, DroneRedAgent, \
#     RandomAgent, BaseAgent, RedMeanderAgent
from CybORG.Agents.SimpleAgents.DroneRedAgent import DroneRedAgent
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator, DroneSwarmScenarioGenerator
import pygame
from CybORG.Simulator.Actions.ConcreteActions.RemoveOtherSessions import RemoveOtherSessions
from CybORG.Simulator.Actions.ConcreteActions.ExploitActions.RetakeControl import RetakeControl
import random
from pprint import pprint
from CybORG.Shared.Results import Results
from CybORG.Agents.Wrappers import PettingZooParallelWrapper

/Users/myong/Library/Caches/pypoetry/virtualenvs/cyborg-b_S6pbUY-py3.9/lib/python3.9/site-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)


In [2]:
num_drones=5
# sg = DroneSwarmScenarioGenerator(num_drones=num_drones, max_length_data_links=25, red_spawn_rate=0, starting_num_red=0)
red_agent = DroneRedAgent
sg = DroneSwarmScenarioGenerator(num_drones=num_drones, default_red_agent=DroneRedAgent, starting_num_red=1)
cyborg = CybORG(scenario_generator=sg, environment='sim',seed=0)

wrapped_cyborg = PettingZooParallelWrapper(env=cyborg)
observations = wrapped_cyborg.reset()

def get_teams(verbose=True)->set:
  teams = cyborg.environment_controller.team
  if verbose:
    print("{} Teams: {}".format(len(teams.keys()), set(teams.keys())))
  return teams

def get_members(team:str, verbose:bool=True)->list:
  teams = cyborg.environment_controller.team
  if verbose: 
    print("\n{} Members: {}".format(team, teams[team]))
  return teams[team]

def get_actions(agent, verbose:bool=True)->list:
  agent_action_space = cyborg.get_action_space(agent)['action']
  filtered_action_space = {key: value for key, value in agent_action_space.items() if value}
  action_classes = list(filtered_action_space.keys())
  action_names = [action.__name__ for action in action_classes]
  if verbose:
    print("{} ({}) action space: {}".format(agent, len(action_names), action_names))
  return action_names

def get_wrapped_actions(agent, verbose:bool=True)->list:
  action_names = wrapped_cyborg.get_action_space(agent)
  if verbose:
    print("{} ({}) action space: {}".format(agent, len(action_names), action_names))
  return action_names

def print_players()->tuple[list[str],list[str]]:
  teams = get_teams()
  for team in teams.keys():
    members = get_members(team)
    for member in members:
      get_wrapped_actions(member)  
  
def get_drone_position(agent:str, verbose = True)->dict[str, list[float]]:
  observations = cyborg.get_observation(agent)
  positions = {}
  for k,v in observations.items():
    if "drone" in k:
      if 'position' in v['System info']:
        positions[k] = [int(p) for p in v['System info']['position']]
  if verbose:
    for k,v in positions.items():
      print("Agent {} thinks that {} is at [{},{}]".format(agent, k, v[0], v[1]) )
  return positions   

print_players()
blues = get_members('Blue', verbose=False)
reds = get_members('Red', verbose=False)

# get_players()

red_observations = cyborg.environment_controller.get_last_observation(reds[0]).get_dict()
# print(red_observations)
blue_observations = cyborg.environment_controller.get_last_observation(blues[0]).get_dict()
assert(set(red_observations.keys())==set(blue_observations.keys()))

3 Teams: {'Blue', 'Red', 'Green'}

Red Members: ['red_agent_0', 'red_agent_1', 'red_agent_2', 'red_agent_3', 'red_agent_4', 'Red_Trojan']
red_agent_0 (26) action space: ['ExploitDroneVulnerability drone_0', 'ExploitDroneVulnerability drone_1', 'ExploitDroneVulnerability drone_2', 'ExploitDroneVulnerability drone_3', 'ExploitDroneVulnerability drone_4', 'SeizeControl drone_0', 'SeizeControl drone_1', 'SeizeControl drone_2', 'SeizeControl drone_3', 'SeizeControl drone_4', 'FloodBandwidth drone_0', 'FloodBandwidth drone_1', 'FloodBandwidth drone_2', 'FloodBandwidth drone_3', 'FloodBandwidth drone_4', 'BlockTraffic drone_0', 'BlockTraffic drone_1', 'BlockTraffic drone_2', 'BlockTraffic drone_3', 'BlockTraffic drone_4', 'AllowTraffic drone_0', 'AllowTraffic drone_1', 'AllowTraffic drone_2', 'AllowTraffic drone_3', 'AllowTraffic drone_4', 'Sleep']
red_agent_1 (26) action space: ['ExploitDroneVulnerability drone_0', 'ExploitDroneVulnerability drone_1', 'ExploitDroneVulnerability drone_2', 'Ex

/Users/myong/Library/Caches/pypoetry/virtualenvs/cyborg-b_S6pbUY-py3.9/lib/python3.9/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


In [3]:
def get_drone_sessions(agents:list[str], verbose=True)->dict[str, str]:
  my_drones = {}
  for a in agents:
    obs = cyborg.get_observation(a)
    obs_about_drones = { k : obs[k]
                 for k in obs
                 if "drone_" in k}
    for d, o in obs_about_drones.items():
      if 'Sessions' in o:
        # pprint(o['Sessions'])
      # [{'Agent': 'blue_agent_0',
      # 'ID': 0,
      # 'Type': <SessionType.BLUE_DRONE_SESSION: 13>,
      # 'Username': 'root'}]
        for s in o['Sessions']:
          # print(s['Agent'])
          if d in my_drones:
            if (isinstance(my_drones[d], list)):
              my_drones[d].append(a)
            else:
              my_drones[d] = [a]
          else:
            my_drones[d] = [a]
  if (verbose):
    for drone, owner in my_drones.items():
      pprint("{} running sessions by {}".format(drone, owner))
  return my_drones

drone_sessions = get_drone_sessions(blues+reds, verbose=True)
# pprint(cyborg.get_observation('red_agent_2'))

"drone_0 running sessions by ['blue_agent_0']"
"drone_1 running sessions by ['blue_agent_1']"
"drone_3 running sessions by ['blue_agent_3']"
"drone_4 running sessions by ['blue_agent_4']"
"drone_2 running sessions by ['red_agent_2']"


In [4]:
# print(wrapped_cyborg.env.environment_controller.get_connected_agents(reds[0]))
# pprint("Blue: \n{}".format(cyborg.get_agent_state("blue_agent_1")))
# pprint(cyborg.get_agent_state("blue_agent_0"))
# cyborg.get_observation("red_agent_0")
# wrapped_cyborg.get_action_space("red_agent_0")



p = get_drone_position("blue_agent_0", verbose=True)
# print("Red: {}".format(cyborg.get_agent_state("red_agent_1")))


Agent blue_agent_0 thinks that drone_0 is at [75,36]
Agent blue_agent_0 thinks that drone_1 is at [99,71]
Agent blue_agent_0 thinks that drone_2 is at [45,8]
Agent blue_agent_0 thinks that drone_3 is at [37,35]
Agent blue_agent_0 thinks that drone_4 is at [79,48]


In [5]:
from CybORG.Evaluation.submission.RandomAgent import RandomAgent

blue_agents = {f"{agent}": RandomAgent() for agent in blues}

def get_team_last_actions(team:str, verbose=True)->dict[str, str]:
    team_actions = {}
    members = get_members(team=team, verbose=False)
    for member in members:
        action = cyborg.get_last_action(member)
        if verbose:
            print("{}: {}".format(member, action))
        team_actions[member]=action
    return team_actions

def one_step(observations:dict, agents:dict[str, RandomAgent], verbose:bool=True)->tuple[dict, dict, dict, dict]:
    actions_taken = {}
    action_spaces = wrapped_cyborg.action_spaces    
    action_index = {
        agent_name: agent.get_action(
            observations[agent_name], 
            action_spaces[agent_name]
        ) 
        for agent_name, agent in agents.items() 
        if agent_name in wrapped_cyborg.agents
    }
    obs, rew, done, info = wrapped_cyborg.step(action_index)

    for k, v in action_index.items():
        actions = get_wrapped_actions(k, False)
        action_taken = str(cyborg.get_last_action(k))
        actions_taken[k]=action_taken
        assert(action_taken in actions[v])
        if (verbose):
            print("{} took action: {}".format(k,actions[v]))
            print("Reward: {}".format(rew[k]))
            if (k in info):
                print("Info: {}".format(info[k]))
            print("Observations: {}".format(cyborg.get_observation(k)))
            print("Done: {}\n".format(done[k]))
    return (obs, rew, done, info)

In [8]:
observations = wrapped_cyborg.reset()
for s in range(30):
    report = False
    if (s%10==0):
        print("\nAt t = {}".format(s))
        get_drone_sessions(blues+reds, verbose=True)
    obs, rew, done, info = one_step(observations, agents=blue_agents,verbose=False)
    # get_team_last_actions(team="Blue", verbose=report)
    # get_team_last_actions(team="Red", verbose=report)
print("\nAt t = {}".format(30))
get_drone_sessions(blues+reds, verbose=True)


At t = 0
"drone_0 running sessions by ['blue_agent_0']"
"drone_1 running sessions by ['blue_agent_1']"
"drone_2 running sessions by ['blue_agent_2']"
"drone_4 running sessions by ['blue_agent_4']"
"drone_3 running sessions by ['red_agent_3']"

At t = 10
"drone_0 running sessions by ['blue_agent_0']"
"drone_2 running sessions by ['blue_agent_2']"
"drone_4 running sessions by ['blue_agent_4']"
"drone_1 running sessions by ['red_agent_1', 'red_agent_1']"
"drone_3 running sessions by ['red_agent_3']"

At t = 20
"drone_0 running sessions by ['blue_agent_0']"
"drone_2 running sessions by ['blue_agent_2']"
"drone_4 running sessions by ['blue_agent_4']"
"drone_1 running sessions by ['red_agent_1', 'red_agent_1']"
"drone_3 running sessions by ['red_agent_3', 'red_agent_3']"

At t = 30
"drone_0 running sessions by ['blue_agent_0']"
"drone_2 running sessions by ['blue_agent_2']"
"drone_4 running sessions by ['blue_agent_4']"
("drone_1 running sessions by ['red_agent_1', 'red_agent_1', 'red_agent

{'drone_0': ['blue_agent_0'],
 'drone_2': ['blue_agent_2'],
 'drone_4': ['blue_agent_4'],
 'drone_1': ['red_agent_1', 'red_agent_1', 'red_agent_1', 'red_agent_1'],
 'drone_3': ['red_agent_3', 'red_agent_3']}

In [14]:
def show_connections(agent:str, verbose=True)->list[str]:
  connections = wrapped_cyborg.env.environment_controller.get_connected_agents(agent)
  connections_no_self = [c
                         for c in connections
                         if c not in agent]
  if verbose:
    print("{} is connected to {}".format(agent, connections_no_self))
  return connections_no_self 

def find_connected_red_drones(agent:str, verbose=True)->list[str]:
  connected_red_drones = []
  connections = show_connections(agent, verbose=False)
  red_drones = get_drone_sessions(reds, verbose=False)
  for drone, agents in red_drones.items():
    connected_red_agents = set(connections).intersection(agents)
    if(len(connected_red_agents)>0):
      connected_red_drones.append(drone)
  if verbose:
    print("{} is connected to {} red drones: {}".format(agent, 
                                                          len(connected_red_agents), 
                                                          connected_red_drones))
  return connected_red_drones

def recommend_an_action(agent:str, verbose=True)->list[str]:
  connected_red_drones = find_connected_red_drones(agent=agent, verbose=False)
  actions = get_wrapped_actions(agent, verbose=False)
  relevant_actions = []
  for drone_name in connected_red_drones:
    for action in actions:
      if drone_name in action:
        relevant_actions.append(action)
  if verbose:
    if (len(relevant_actions)>0):
      print("{} is connected to Red's {}.\nPlausible actions are {}".format(agent, connected_red_drones, relevant_actions))
  return relevant_actions

# recommend_an_action(blues[0])


In [23]:
from CybORG.Simulator.Actions import *

def get_team_last_actions(team:str, verbose=True)->list:
    team_actions = []
    members = get_members(team=team, verbose=False)
    for member in members:
        action = cyborg.get_last_action(member)
        if verbose:
            print("{}: {}".format(member, action))
        team_actions.append(action)
    return team_actions

def random_step(agents:list, verbose:bool=True)->tuple[dict, dict, dict, dict, dict]:
    action_index={}
    action_names={}
    actions_taken = {}
    for a in agents:
        actions = get_wrapped_actions(agent=a, verbose=False)
        action_index[a] = random.choice(range(0,len(actions)-1))
        action_names[a] = actions[action_index[a]]

    obs, reward, done, info = wrapped_cyborg.step(action_index)
    
    for k, v in action_index.items():
        actions = get_wrapped_actions(k, False)
        action_taken = str(cyborg.get_last_action(k))
        actions_taken[k]=action_taken
        assert(action_taken in actions[v])
        if verbose:
            print("{} took action: {}".format(k,actions[v]))
            print("Reward: {}".format(rew[k]))
            if (k in info):
                print("Info: {}".format(info[k]))
            print("Observations: {}".format(cyborg.get_observation(k)))
            print("Done: {}\n".format(done[k]))

    return obs, reward, done, info
    #     clock.tick(frame_rate)
    
def human_step(agents:list, verbose:bool=True)->tuple[dict, dict, dict, dict, dict]:
    action_index={}
    action_names={}
    actions_taken = {}
    for a in agents:        
        if a in wrapped_cyborg.agents:
            actions = get_wrapped_actions(agent=a, verbose=False)
            plausible_actions = recommend_an_action(agent=a,verbose=True)
            plausible_actions.append("Sleep")
            for choice in plausible_actions:
                print("Press {} for {}".format(actions.index(choice), choice))
            choice = input("Make your choice [0-{}]. q to quit.".format(len(actions)-1))
            choice_int = int(choice)
            if not isinstance(choice_int, int):
                break
            action_index[a] = choice_int
            action_names[a] = actions[choice_int]

    # print(action_index)
    obs, reward, done, info = wrapped_cyborg.step(action_index)
    
    for k, v in action_index.items():
        actions = get_wrapped_actions(k, False)
        action_taken = str(cyborg.get_last_action(k))
        actions_taken[k]=action_taken
        assert(action_taken in actions[v])
        if verbose:
            print("{} took action: {}".format(k,actions[v]))
            if (k in reward):
                print("Reward: {}".format(reward[k]))
            if (k in info):
                print("Info: {}".format(info[k]))
            print("Observations: {}".format(cyborg.get_observation(k)))
            print("Done: {}\n".format(done[k]))
        
    return (obs, reward, done, info)

wrapped_cyborg.reset()
get_drone_sessions(blues+reds, True)
obs, reward, done, info = human_step(blues,verbose=True)
get_drone_sessions(blues+reds, True)
# obs, reward, done, info = random_step(blues,verbose=True)
# get_team_last_actions(team="Blue", verbose=report)
# get_team_last_actions(team="Red", verbose=report)
# get_my_drones(blues+reds, verbose=True)

"drone_0 running sessions by ['blue_agent_0']"
"drone_1 running sessions by ['blue_agent_1']"
"drone_3 running sessions by ['blue_agent_3']"
"drone_4 running sessions by ['blue_agent_4']"
"drone_2 running sessions by ['red_agent_2']"
Press 16 for Sleep


Press 16 for Sleep
Press 16 for Sleep
blue_agent_4 is connected to Red's ['drone_2'].
Plausible actions are ['RetakeControl drone_2', 'BlockTraffic drone_2', 'AllowTraffic drone_2']
Press 2 for RetakeControl drone_2
Press 8 for BlockTraffic drone_2
Press 13 for AllowTraffic drone_2
Press 16 for Sleep
blue_agent_0 took action: Sleep
Reward: -4.0
Observations: {'success': <TrinaryEnum.UNKNOWN: 2>, 'drone_0': {'System info': {'Hostname': 'drone_0', 'position': array([46.88745224, 91.25490034])}, 'Interface': [{'IP Address': IPv4Address('10.0.43.215'), 'Subnet': IPv4Network('10.0.43.208/28'), 'blocked_ips': [], 'NetworkConnections': []}], 'Sessions': [{'Username': 'root', 'ID': 0, 'Type': <SessionType.BLUE_DRONE_SESSION: 13>, 'Agent': 'blue_agent_0'}]}, 'drone_1': {'System info': {'Hostname': 'drone_1'}, 'Interface': [{'IP Address': IPv4Address('10.0.43.217'), 'Subnet': IPv4Network('10.0.43.208/28')}]}, 'drone_2': {'System info': {'Hostname': 'drone_2'}, 'Interface': [{'IP Address': IPv4Ad

{'drone_0': ['blue_agent_0'],
 'drone_1': ['blue_agent_1'],
 'drone_2': ['blue_agent_2'],
 'drone_3': ['blue_agent_3'],
 'drone_4': ['blue_agent_4']}

In [ ]:
def show_connections(agent:str, verbose=True)->list[str]:
  connections = wrapped_cyborg.env.environment_controller.get_connected_agents(agent)
  connections_no_self = [c
                         for c in connections
                         if c not in agent]
  if verbose:
    print("{} is connected to {}".format(agent, connections_no_self))
  return connections_no_self 

def find_connected_red_drones(agent:str, verbose=True)->list[str]:
  connected_red_drones = []
  connections = show_connections(agent, verbose=False)
  red_drones = get_my_drones(reds, verbose=False)
  for drone, agents in red_drones.items():
    connected_red_agents = set(connections).intersection(agents)
    if(len(connected_red_agents)>0):
      connected_red_drones.append(drone)
  if verbose:
    print("{} is connected to {} red drones: {}".format(agent, 
                                                          len(connected_red_agents), 
                                                          connected_red_drones))
  return connected_red_drones

def recommend_an_action(agent:str, verbose=True)->list[str]:
  connected_red_drones = find_connected_red_drones(agent=agent, verbose=False)
  actions = get_wrapped_actions(agent, verbose=False)
  relevant_actions = []
  for drone_name in connected_red_drones:
    for action in actions:
      if drone_name in action:
        relevant_actions.append(action)
  if verbose:
    print("{} is connected to {}.\nPlausible actions are {}".format(agent, connected_red_drones, relevant_actions))
  return relevant_actions
    
recommend_an_action(blues[0])

In [ ]:
from PIL import Image, ImageDraw
height = 1000
width = 1000
num_cols = 100
num_rows = 100

def draw_grid(num_rows, num_cols)->Image:
  image = Image.new(mode='L', size=(height, width), color=255)
  # Draw some lines
  draw = ImageDraw.Draw(image)
  y_start = 0
  y_end = image.height
  x_start = 0
  x_end = image.width

  step_size_col = int(image.width / num_cols)
  step_size_row = int(image.height / num_rows)

  for x in range(0, image.width, step_size_col):
      line = ((x, x_start), (x, x_end))
      draw.line(line, fill=128)

  for y in range(0, image.height, step_size_row):
      line = ((y_start, y), (y_end,y))
      draw.line(line, fill=128)

  del draw
  return image

def draw_drones(background:Image, x:int, y:int)->Image:
   drone_image = Image.open("blue_drone.png")
   drone_width = int(width / num_cols)
   drone_height = int(height / num_cols)
   drone_resized = drone_image.resize((drone_width, drone_height))
   background.paste(drone_resized, (x,y))
   return background
   
grid = draw_grid(num_cols,num_rows)
state_of_play = draw_drones(grid, 0,0)
display(state_of_play)

In [ ]:
frame_rate = 15
for i in range(10):
    print("\nEpisode {}".format(i))
    print(76*'-')
    for i in range(10):
        print("\nTurn {}".format(i))
        get_team_last_actions(team="Red")
        get_team_last_actions(team="Blue")
        # stop = cyborg.render(mode='human')
        # if stop:
        #     break
        step(reds, verbose=True)
        step(blues, verbose=True)
        sleep(0.1)
        # a = input(f'Step {i}, use q to quit, use n to go to next demo')
        # if 'q' in a:
        #     quit()
        # if 'n' in a:
        #     break
        # sleep(0.5)
        # clock.tick(frame_rate)
    cyborg.reset()
# a = input('red drone agent')
# a = 'q'
# if 'q' in a:
#     quit()

In [ ]:
frame_rate = 15
for i in range(2):
    print("\nEpisode {}".format(i))
    print(76*'-')
    for i in range(2):
        print("\nTurn {}".format(i))
        get_team_last_actions(team="Red")
        get_team_last_actions(team="Blue")
        # stop = cyborg.render(mode='human')
        # if stop:
        #     break
        step(reds, verbose=True)
        step(blues, verbose=True)
        sleep(0.1)
        # a = input(f'Step {i}, use q to quit, use n to go to next demo')
        # if 'q' in a:
        #     quit()
        # if 'n' in a:
        #     break
        # sleep(0.5)
        # clock.tick(frame_rate)
    cyborg.reset()
# a = input('red drone agent')
# a = 'q'
# if 'q' in a:
#     quit()

In [ ]:


def step(agents:str, verbose:bool=True)->tuple[dict, dict, dict, dict, dict]:
    action_indices={}
    action_names={}
    for a in agents:
        action_space = env.get_action_space(a) 
        action_indices[a] = random.choice(range(0,len(action_space)-1))
        action_names[a] = action_space[action_indices[a]]
    
    obs, reward, done, info = cyborg.parallel_step(action_indices)
    
    if (verbose):
        for k, v in action_names.items():
            print("{}\t:{}".format(k,v))
        print(reward)
        print(obs)
        print(done)
        print(info)

    return action_names, obs, reward, done, info
    #     clock.tick(frame_rate)
    

action_taken, obs, reward, done, info = step(reds,verbose=True)
print(env.get_last_actions(reds[2]))
print(env.get_last_actions(blues[2]))

# red_obs = results.observation
# red_obs = results.observation

In [ ]:
from CybORG.Agents import B_lineAgent
agent = B_lineAgent()

action_space = results.action_space
print(action_space)

# def step_red(obs, verbose=True):
#     action = agent.get_action(obs,action_space)
#     results = env.step(action=action,agent='Red')
#     obs = results.observation
    
#     if verbose:
#         print('Red Action:',action)
#         print(76*'-')
#         pprint(obs)
    
#     return results

In [ ]:
num_drones=5
sg = DroneSwarmScenarioGenerator(num_drones=num_drones, max_length_data_links=25, red_spawn_rate=0, starting_num_red=0)
cyborg = CybORG(sg, 'sim')

clock = pygame.time.Clock()
frame_rate = 15

for i in range(2):
    action_red = cyborg.environment_controller.get_last_action('Red')
    print("Red action: {}".format(action_red))
    # stop = cyborg.render()
    # if stop:
    #     break
    cyborg.step()
    for agent in cyborg.active_agents:
        # if 'red' in agent:
        #     print(agent, cyborg.get_last_action(agent))
        print("Agent: {}".format(agent))
        action_space = cyborg.get_action_space(agent=agent)
        action = agent.get_action(obs, action_space)
        cyborg.step(agent=agent, action=action)
    # sleep(0.1)
    # a = input(f'Step {i}, use q to quit, use n to go to next demo')
    # if 'q' in a:
    #     quit()
    # if 'n' in a:
    #     break
    # sleep(0.5)
    clock.tick(frame_rate)
cyborg.reset()

In [ ]:
sg = DroneSwarmScenarioGenerator(num_drones=num_drones, max_length_data_links=25, red_spawn_rate=0, starting_num_red=0)
cyborg = CybORG(sg, 'sim')

results = env.reset(agent='Red')
observations_red = results.observation

# for i in range(2):
#     action_red = cyborg.environment_controller.get_last_action('Red')
#     print("Red action: {}".format(action_red))

In [ ]:
def get_players()->tuple[list[str],list[str]]:
  print("Players:")
  print(76*'-')
  
  blues = env.active_agents
  print("{} Blues: {}".format(len(blues), blues))
  
  blue_name = blues[0]
  blue_action_space = env.get_action_space(blue_name)
  print("{} ({}) actions: {}".format(blue_name, len(blue_action_space), blue_action_space))

  agents = list(cyborg.environment_controller.agent_interfaces.keys())
  reds=list(filter(lambda a_name: "red" in a_name, agents))
  print("\n{} Reds: {}".format(len(reds),reds))
  red_name = reds[0]
  red_action_space = env.get_action_space(red_name)
  print("{} ({}) actions: {}".format(red_name, len(red_action_space), red_action_space))

  greens=list(filter(lambda a_name: "green" in a_name, agents))
  print("\n{} Greens: {}".format(len(greens),greens))
  green_name = greens[0]
  green_action_space = env.get_action_space(green_name)
  print("{} ({}) actions: {}".format(green_name, len(green_action_space), green_action_space))

  # print(cyborg.environment_controller.)
  return blues, reds